In [1]:
import torch
import gpytorch
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from gpytorch.mlls import SumMarginalLogLikelihood

In [2]:
DATA_FILEPATH = "/home/hansm/active_learning/Double_pendulum/data/"
file_path_train_inputs = DATA_FILEPATH + 'train_inputs.csv'
df_train_inputs = pd.read_csv(file_path_train_inputs)
train_inputs = df_train_inputs.values

file_path_train_outputs = DATA_FILEPATH + 'train_outputs.csv'
df_train_outputs = pd.read_csv(file_path_train_outputs)
train_outputs = df_train_outputs.values

# Set the number of rows you want to choose
num_rows_to_choose = 1000

# Choose 2000 random indices
random_indices = np.random.choice(train_inputs.shape[0], size=num_rows_to_choose, replace=False)

# Select the corresponding rows from each array
train_inputs = train_inputs[random_indices]
train_outputs = train_outputs[random_indices]

train_inputs = torch.tensor(train_inputs, dtype=torch.float32)

In [3]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [4]:
def train_gp(training_inputs, training_outputs, training_iterations):
    gaussian_likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model_list = []
    for col_index in range(training_outputs.shape[1]):
        training_outputs_column = torch.tensor(training_outputs[:, col_index], dtype=torch.float32)
        exact_model = ExactGPModel(training_inputs, training_outputs_column, gaussian_likelihood)
        model_list.append(exact_model)
    likelihoods = [model.likelihood for model in model_list]
    likelihood = gpytorch.likelihoods.LikelihoodList(*likelihoods)
    model = gpytorch.models.IndependentModelList(*model_list)

    mll = SumMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    # Use the Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(*model.train_inputs)
        loss = -mll(output, model.train_targets)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()

    return model, likelihood

In [ ]:
gp_model, gp_likelihood = train_gp(train_inputs, train_outputs, 500)

In [12]:
print(type(gp_model))

<class 'gpytorch.models.model_list.IndependentModelList'>


In [13]:
def gp_eval(test_inputs, model, likelihood):
  test_inputs = torch.tensor(test_inputs, dtype=torch.float32)
  # Set into eval mode
  model.eval()
  likelihood.eval()


  # Make predictions (use the same test points)
  with torch.no_grad(), gpytorch.settings.fast_pred_var():
      
      # This contains predictions for both outcomes as a list
      predictions = likelihood(*model(test_inputs, test_inputs, test_inputs, test_inputs))
      predictions_lst = [prediction.mean.numpy() for prediction in predictions]
      final_prediction = np.column_stack((predictions_lst))
      variance_lst = []
      for prediction in predictions:
        lower, upper = prediction.confidence_region()
        variance = upper.numpy() - lower.numpy()
        variance_lst.append(variance)
  return final_prediction, np.column_stack(variance_lst)

In [14]:
file_path_test_inputs = DATA_FILEPATH + 'test_inputs.csv'
df_test_inputs = pd.read_csv(file_path_test_inputs)
test_inputs = df_test_inputs.values

file_path_test_outputs = DATA_FILEPATH + 'test_outputs.csv'
df_test_outputs = pd.read_csv(file_path_test_outputs)
test_outputs = df_test_outputs.values

test_inputs = torch.tensor(test_inputs, dtype=torch.float32)

In [15]:
prediction, var = gp_eval(test_inputs, gp_model, gp_likelihood)

/tmp/ipykernel_1215078/681682463.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_inputs = torch.tensor(test_inputs, dtype=torch.float32)


In [10]:
print(prediction.shape)

(6561, 4)


In [16]:
RMSE = np.sqrt(mean_squared_error(test_outputs, prediction))
print(RMSE)

0.12079695030002702
